### Example execution of [FAIRsoft evaluator web app](https://openebench.bsc.es/observatory/Evaluation/) via FAIRsoft APIs

This notebook is intended to provide an example and validation for using FAIRsoft APIs to extract, score and evaluate the FAIRness of research software metadata in a GitHub repository. There are other features of the FAIRsoft tool (e.g., metadata updating) that are not included in this notebook.

In [1]:
# import custom libraries
import preprocess_utils
# these libaries handle making API requests
import extractor_client
import metadata_client
import evaluation_client

A GitHub URL can be used as input to this process. Note that the [Metadata Extractor for FAIRsoft](https://github.com/settings/installations/54726444) GitHub App must be installed in the user workspace with read access to the input repository.

In [2]:
github_url = 'https://github.com/abnerbog/hydroshare'

We can parse the repository and owner names for feeding into the [GitHub Metadata Extractor API](https://github.com/inab/github-metadata-api/tree/main).

In [3]:
repo,owner = preprocess_utils.get_github_repo_and_owner(github_url)

If the GitHub App is installed, an installation ID will be returned that can be used to extract repository metadata.

In [4]:
install_id = extractor_client.get_installation_id(repo,owner)

In [5]:
metadata = metadata_client.get_repository_metadata(repo,owner,install_id)

The JSON response can be parsed to return the metadata as a dictionary:

In [6]:
print(metadata)

{'name': 'hydroshare', 'label': ['hydroshare'], 'description': [{'term': 'HydroShare is a collaborative website for better access to data and models in the hydrologic sciences. ', 'id': 0}], 'links': [], 'webpage': [{'term': 'https://www.hydroshare.org', 'id': 0}], 'isDisabled': False, 'isEmpty': False, 'isLocked': False, 'isPrivate': False, 'isTemplate': False, 'version': [], 'license': [{'term': {'name': 'BSD 3-Clause "New" or "Revised" License', 'url': 'http://choosealicense.com/licenses/bsd-3-clause/'}, 'id': 0}], 'repository': [{'term': 'https://github.com/abnerbog/hydroshare', 'id': 0}], 'topics': [], 'operations': [], 'authors': [{'term': {'name': 'Devin Cowan', 'type': 'person', 'email': 'devincowan@users.noreply.github.com', 'maintainer': False}, 'id': 0}, {'term': {'name': 'Devin Cowan', 'type': 'person', 'email': 'dcowan@cuahsi.org', 'maintainer': False}, 'id': 1}, {'term': {'name': 'Scott Black', 'type': 'person', 'email': 'sblack@cuahsi.org', 'maintainer': False}, 'id': 2}

The evaluation tool can be run by passing the metadata as a payload to the [Software Observatory API](https://github.com/inab/observatory-api).

In [7]:
results,logs = evaluation_client.get_fairsoft_results_and_logs(metadata)

The results are parsed and reported below (note [this documentation](https://inab.github.io/FAIRsoft_indicators/) contains details on each of the indicators):

In [8]:
print(results)

{'name': 'hydroshare', 'type': '', 'version': None, 'F': 0.72, 'F1': 0.8, 'F1_1': True, 'F1_2': False, 'F2': 0.6, 'F2_1': True, 'F2_2': False, 'F3': 0.7, 'F3_1': False, 'F3_2': True, 'F3_3': False, 'A': 0.0, 'A1': 0.0, 'A1_1': False, 'A1_2': False, 'A1_3': False, 'A1_4': False, 'A1_5': False, 'A2': 0.0, 'A2_1': False, 'A2_2': False, 'A3': 0.0, 'A3_1': False, 'A3_2': False, 'A3_3': False, 'A3_4': False, 'A3_5': False, 'I': 0.0, 'I1': 0.0, 'I1_1': False, 'I1_2': False, 'I1_3': False, 'I1_4': False, 'I1_5': False, 'I2': 0.0, 'I2_1': False, 'I2_2': False, 'I3': 0.0, 'I3_1': False, 'I3_2': False, 'I3_3': False, 'R': 0.5, 'R1': 0.0, 'R1_1': False, 'R1_2': False, 'R2': 1.0, 'R2_1': True, 'R2_2': True, 'R3': 1.0, 'R3_1': False, 'R3_2': True, 'R4': 0.0, 'R4_1': False, 'R4_2': False, 'R4_3': False}


Lastly, a pass/fail evaluation of the results can also be reported in the logs:

In [9]:
print(logs)

{'F1_1': ['The metadata is assigned a name to be identified.'], 'F1_2': ['❌ No version provided.', 'Result: FAILED'], 'F2_1': ['⚙️ Checking if any source provides structured metadata.', "🔍 Sources provided: ['github']", '⚙️ Comparing sources against: ', "✅ Source 'github' provides structured metadata.", 'Result: PASSED'], 'F2_2': ['⚙️ Verifying whether a topic or operation is part of a specific ontology or controlled vocabulary.', '🔍 No topics provided.', '🔍 No operations provided.', '❌ No topic or operation involves the use of ontologies or controlled vocabularies.', 'Result: FAILED'], 'F3_1': ['⚙️ Checking if any source matches software registries.', "🔍 Sources provided: ['github']", " ⚙️ Comparing sources against: ['biotools', 'bioconda', 'bioconductor', 'toolshed', 'sourceforge']", '❌ No source matches software registries. Checking registries ...', '⚙️ Checking if the specified software registry matches any of the known registries.', '🔍 No registries provided.', '⚙️ Comparing regis